## **EXERCÍCIO 10 - Agents**

## **CESAR SCHOOL**
* Pós-graduação em Engenharia e Análise de Dados - 2023.2
* **Disciplina: Tópicos Complementares**
* Professor: **Silvan Ferreira**
* Aluno: **Allan Bispo** - apsb@cesar.school

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Success

In [5]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.8 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [13]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(filename: str, content: str) -> None:
    """Escreve conteúdo em um arquivo"""
    with open(filename, "w") as f:
        f.write(content)

@tool
def count_letters(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra em uma palavra"""
    return word.lower().count(letter.lower())

tools = [weather, multiply, write_file, count_letters]
llm_with_tools = llm.bind_tools(tools)

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [15]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


neveO clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical.

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical.'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

* RESPOSTA: foi adicionado ao tool a função

          def write_file(filename: str, content: str) -> None:
              """Escreve conteúdo em um arquivo"""
            with open(filename, "w") as f:
              f.write(content)

In [10]:
agent_executor.invoke({"input": "Crie um arquivo de texto sobre fiba 3x3"})



> Entering new AgentExecutor chain...

Invoking: `write_file` with `{'filename': 'fiba_3x3.txt', 'content': 'FIBA 3x3 é uma modalidade de basquete que se destaca por sua dinâmica e formato único. Criada pela Federação Internacional de Basquete (FIBA), essa versão do esporte é jogada em uma quadra reduzida, com apenas três jogadores em cada equipe, e tem ganhado popularidade em todo o mundo.\n\n### Regras Básicas:\n- **Duração do Jogo:** As partidas são jogadas em um tempo de 10 minutos ou até que uma equipe alcance 21 pontos.\n- **Pontuação:** Os pontos são contados de forma diferente do basquete tradicional. Um arremesso de dentro do arco vale 1 ponto, enquanto um arremesso de fora do arco vale 2 pontos. Lances livres valem 1 ponto.\n- **Faltas:** Após a 7ª falta de uma equipe, os adversários recebem lances livres.\n\n### Competição:\nFIBA 3x3 organiza torneios internacionais, incluindo a Copa do Mundo de 3x3 e eventos nos Jogos Olímpicos. A modalidade é conhecida por seu ritmo acel

{'input': 'Crie um arquivo de texto sobre fiba 3x3',
 'output': 'O arquivo de texto sobre FIBA 3x3 foi criado com sucesso. Se precisar de mais alguma coisa, é só avisar!'}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

* RESPOSTA: foi adicionado ao tool a função


    def count_letters(word: str, letter: str) -> int:
        """Conta a quantidade de ocorrências de uma letra em uma palavra"""
        return word.count(letter)

In [16]:
agent_executor.invoke({"input": "quantas letras N há na palavra inconstitucionalissimamente?"})



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'word': 'inconstitucionalissimamente', 'letter': 'n'}`


4Na palavra "inconstitucionalissimamente", há 4 letras "N".

> Finished chain.


{'input': 'quantas letras N há na palavra inconstitucionalissimamente?',
 'output': 'Na palavra "inconstitucionalissimamente", há 4 letras "N".'}